# marcus johnson  week 5 hw 

In [1]:
!conda install numpy scipy scikit-learn pandas joblib pytorch

^C


In [2]:
!pip install deap update_checker tqdm stopit xgboost

^C


In [3]:
!pip install tpot

^C


In [8]:
!conda install pytorch-cpu torchvision-cpu -c pytorch

^C


In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import timeit 

In [52]:
import pandas as pd
data = pd.read_csv('cleaned_Churn_data.csv', index_col='customerID')
data.head()

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure
customerID,,,,,,,,
7590-VHVEG,1.0,0,0,3,29.85,29.85,0,29.850000
5575-GNVDE,34.0,1,1,2,56.95,1889.50,0,55.573529
3668-QPYBK,2.0,1,0,2,53.85,108.15,1,54.075000
7795-CFOCW,45.0,0,1,1,42.30,1840.75,0,40.905556
9237-HQITU,2.0,1,0,3,70.70,151.65,1,75.825000


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tenure             7043 non-null   float64
 1   PhoneService       7043 non-null   int64  
 2   Contract           7043 non-null   int64  
 3   PaymentMethod      7043 non-null   int64  
 4   MonthlyCharges     7043 non-null   float64
 5   TotalCharges       7043 non-null   float64
 6   Churn              7043 non-null   int64  
 7   charge_per_tenure  7043 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 495.2+ KB


In [54]:
features = data.drop('Churn', axis=1)
targets = data['Churn']

x_train, x_test, y_train, y_test = train_test_split(features, targets, stratify=targets, random_state=42)

In [55]:
%%time
tpot = TPOTClassifier(generations=5, population_size=50, cv=5,random_state=42, scoring='accuracy', verbosity=2, n_jobs=-1)

tpot.fit(x_train, y_train)
print(tpot.score(x_test, y_test))

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7976100885869097

Generation 2 - Current best internal CV score: 0.7976100885869097

Generation 3 - Current best internal CV score: 0.7976100885869097

Generation 4 - Current best internal CV score: 0.7976100885869097

Generation 5 - Current best internal CV score: 0.7977975115392334

Best pipeline: LogisticRegression(input_matrix, C=5.0, dual=False, penalty=l2)
0.7950028392958546
CPU times: total: 35.8 s
Wall time: 16min 14s


this is showing that my best pipeline was in the LR model 

In [56]:
predictions = tpot.predict(x_test)
predictions

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [57]:
print('Predictions for test data set')
print(predictions)
print('Actuals for test data set')
print(y_test)

Predictions for test data set
[0 0 0 ... 0 1 0]
Actuals for test data set
customerID
5343-SGUBI    0
5442-BXVND    0
6434-TTGJP    0
1628-BIZYP    0
0298-XACET    0
             ..
3780-DDGSE    0
4154-AQUGT    1
1116-DXXDF    0
3237-AJGEH    1
6704-UTUKK    0
Name: Churn, Length: 1761, dtype: int64


My actuals are linning up with my prediction test shown above.

In [58]:
from sklearn.metrics import accuracy_score
print(f'Accuracy of the TPOT predictions: {accuracy_score(y_test,predictions)}')

Accuracy of the TPOT predictions: 0.7950028392958546


In the cell above shows my actual score from my predition model.

In [71]:
tpot.export('tpot_Churn_pipeline_2.py')

In [79]:
from IPython.display import Code

Code('tpot_Churn_pipeline_2.py')

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from tpot.export_utils import set_param_recursive
# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('cleaned_Churn_data.csv', index_col='customerID')
tpot_data.rename(columns={'Churn': 'target'})
features = tpot_data.drop('Churn', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Churn'], random_state=42)

# Average CV score on the training set was: 0.7977975115392334
exported_pipeline = LogisticRegression(C=5.0, dual=False, penalty="l2", max_iter=5000)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

here I altered my code above 

In [80]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=5000)

In [82]:
%run tpot_Churn_pipeline_2.py

[1 0 0 ... 0 1 0]


In [83]:
predictions

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In summary throughout this entire notebook the goal was to be able to generate and make predictions off of a data set. We did that and also were able to get the actual test as well so we could compare and see if our preditions were close or if they were off. Judging by the stats in the data set they were not spot on but they were no far apart. 